In [14]:
#Inserire in una cartella di lavoro i file zip dei dataset
#Creazione delle directory ed estrazione dei dataset

import os
path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)
cwd = os.getcwd()
print(cwd)

#Crea directory per estrarre i dataset solo una volta all'inizio
if not os.path.exists('celeba'):
    os.makedirs('celeba')
if not os.path.exists('utkface'):
    os.makedirs('utkface')

#Estrae i due dataset solo una volta all'inizio
from zipfile import ZipFile
with ZipFile('celeba-dataset.zip', 'r') as zipObj:
    if len(os.listdir('celeba')) == 0:
        zipObj.extractall('celeba')

with ZipFile('utkface-new.zip', 'r') as zipObj:
    if len(os.listdir('utkface')) == 0:
        zipObj.extractall('utkface')

C:\Users\vince\Desktop\Progetto FVAB


In [73]:
import shutil,os,csv
path=r"C:\Users\vince\Desktop\Progetto FVAB"
os.chdir(path)
cwd = os.getcwd()
print(cwd)

created=False
if not os.path.exists('merged_dataset') and not created:
    os.makedirs('merged_dataset')
    created=True
os.chdir('merged_dataset')

if not os.path.exists('images'):
    os.makedirs('images')

cwd = os.getcwd()

path_images_celeba=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba"
path_images_celeba2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\img_align_celeba\img_align_celeba2"

path_images_utkface=r"C:\Users\vince\Desktop\Progetto FVAB\utkface\UTKFace"
path_images_utkface2=r"C:\Users\vince\Desktop\Progetto FVAB\\utkface\UTKFace2"

path_celeba_csv=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba.csv"
path_celeba_csv2=r"C:\Users\vince\Desktop\Progetto FVAB\celeba\list_attr_celeba2.csv"

dest_path=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset\images"
dest_path_oldnew_csv=r"C:\Users\vince\Desktop\Progetto FVAB\merged_dataset"

print(os.listdir(path_images_celeba2))
i=0


#Parte di CelebA
#Copia le immagini in merged_dataset\images e crea un file csv dove mappo oldname_newname
with open(path_celeba_csv2) as csv_file, \
    open(os.path.join(dest_path_oldnew_csv,"oldname_newname.csv"), mode='w',newline="") as name_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    name_writer = csv.writer(name_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)

    line_count = 0

    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            print(row[21]) #Colonna MALE : 1 è uomo, -1 è donna
            line_count += 1
            
            name_writer.writerow(['Old_Name', 'New_Name'])

        else:
            print(f'\t{row[0]}, SEX {row[21]} .')
            line_count += 1
            
            if(row[21]=="1"):
                new_file_name= '%06d_1.jpg'%(line_count-1,)
            else:
                new_file_name= '%06d_0.jpg'%(line_count-1,)

            shutil.copy(os.path.join(path_images_celeba2,row[0]),os.path.join(dest_path, new_file_name))
            #creare csv nomevecchio nome nuovo
            name_writer.writerow([row[0],new_file_name])
   
    print(f'Processed {line_count} lines.')
    
    for filename in os.listdir(path_images_utkface2):
        #print(filename)
        items = filename.split('_')
        line_count += 1
            
        if(items[1]=="0"):#in UTKFace 0 è uomo
            new_file_name= '%06d_1.jpg'%(line_count-1,)
        else:
              new_file_name= '%06d_0.jpg'%(line_count-1,)

        shutil.copy(os.path.join(path_images_utkface2,filename),os.path.join(dest_path, new_file_name))
        #creare csv nomevecchio nome nuovo
        name_writer.writerow([filename,new_file_name])
    
#Parte di UTKFace vedere cosa scegliere attento al nome dei file


C:\Users\vince\Desktop\Progetto FVAB
['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg']
Column names are image_id, 5_o_Clock_Shadow, Arched_Eyebrows, Attractive, Bags_Under_Eyes, Bald, Bangs, Big_Lips, Big_Nose, Black_Hair, Blond_Hair, Blurry, Brown_Hair, Bushy_Eyebrows, Chubby, Double_Chin, Eyeglasses, Goatee, Gray_Hair, Heavy_Makeup, High_Cheekbones, Male, Mouth_Slightly_Open, Mustache, Narrow_Eyes, No_Beard, Oval_Face, Pale_Skin, Pointy_Nose, Receding_Hairline, Rosy_Cheeks, Sideburns, Smiling, Straight_Hair, Wavy_Hair, Wearing_Earrings, Wearing_Hat, Wearing_Lipstick, Wearing_Necklace, Wearing_Necktie, Young
Male
	000001.jpg, SEX -1 .
	000002.jpg, SEX -1 .
	000003.jpg, SEX 1 .
	000004.jpg, SEX -1 .
	000005.jpg, SEX -1 .
Processed 6 lines.


In [23]:
import os
path=r"C:\Users\vince\Desktop\Progetto FVAB\celeba"
os.chdir(path)
cwd = os.getcwd()
print(cwd)

import pandas as pd
test=pd.read_csv('list_eval_partition.csv')
test.info()

C:\Users\vince\Desktop\Progetto FVAB\celeba
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202599 entries, 0 to 202598
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   image_id   202599 non-null  object
 1   partition  202599 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [24]:
import os
path=r"C:\Users\vince\Desktop\Progetto FVAB\celeba"
os.chdir(path)
cwd = os.getcwd()
print(cwd)

import pandas as pd
test=pd.read_csv('list_landmarks_align_celeba.csv')
test.info()

C:\Users\vince\Desktop\Progetto FVAB\celeba
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202599 entries, 0 to 202598
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_id      202599 non-null  object
 1   lefteye_x     202599 non-null  int64 
 2   lefteye_y     202599 non-null  int64 
 3   righteye_x    202599 non-null  int64 
 4   righteye_y    202599 non-null  int64 
 5   nose_x        202599 non-null  int64 
 6   nose_y        202599 non-null  int64 
 7   leftmouth_x   202599 non-null  int64 
 8   leftmouth_y   202599 non-null  int64 
 9   rightmouth_x  202599 non-null  int64 
 10  rightmouth_y  202599 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 17.0+ MB
